In [13]:
from datetime import datetime
import pandas as pd
import pickle

In [6]:
from sklearn.metrics import root_mean_squared_error

In [3]:
from mlflow.entities import ViewType

In [4]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

2025/07/22 16:01:17 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2025/07/22 16:01:17 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


In [5]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [32]:
client.search_experiments(3)

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/2', creation_time=1753197800771, experiment_id='2', last_update_time=1753197800771, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1752768385078, experiment_id='1', last_update_time=1752768385078, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1752768161030, experiment_id='0', last_update_time=1752768247258, lifecycle_stage='deleted', name='Default', tags={}>]

In [ ]:
client.create_experiment(name="my-cool-experiment")

'2'

In [33]:
runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 6.8",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [34]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: cdae98eb6385475f87ab45f06e7f16ca, rmse: 6.4425
run id: 39aa3c497c4f4ccda1b7f187e3e62e8a, rmse: 6.5321
run id: bf17007a773f499c8a402576d1765ad0, rmse: 6.5824


In [ ]:
run_id = "5ab030db43464dc1a5a8fef74251c91f"
model_uri = f"runs:/{run_id}/models_mlflow_lr"

mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...


MlflowException: Run with id=b89d5beddee242f2b78360cd480fe011 not found

In [37]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 5, stage: None


/tmp/ipykernel_59268/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [ ]:
model_version = 3
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was updated on {datetime.today().date()}"
)

<ModelVersion: aliases=[], creation_timestamp=1753199086289, current_stage='None', deployment_job_state=None, description='The model version 3 was updated on 2025-07-22', last_updated_timestamp=1753199715965, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='acb0224c0421483b8a47050d7b77cc23', run_link=None, source='models:/m-f7f945fb7c3b499ab733c9f63026dfab', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [26]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}")
    y_pred = model.predict(X_test)
    return {"rmse": root_mean_squared_error(y_test, y_pred)}

In [8]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [12]:
client.download_artifacts(run_id=run_id, path="preprocessor", dst_path='.')

'/workspaces/mlops-zoomcamp/02-experiment-tracking/preprocessor'

In [14]:
with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [15]:
X_test = preprocess(df, dv)

In [16]:
target = "duration"
y_test = df[target].values

In [ ]:
%time test_model(name="m-b89d5beddee242f2b78360cd480fe011", X_test=X_test, y_test=y_test)

MlflowException: Logged model with ID 'models_mlflow_lr' not found.

In [47]:
client.search_logged_models(experiment_ids=["1"])

[LoggedModel(artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/models/m-b89d5beddee242f2b78360cd480fe011/artifacts', creation_timestamp=1753200638144, experiment_id='1', last_updated_timestamp=1753200641247, model_id='m-b89d5beddee242f2b78360cd480fe011', model_type=None, model_uri='models:/m-b89d5beddee242f2b78360cd480fe011', name='models_mlflow_lr', source_run_id='5ab030db43464dc1a5a8fef74251c91f', status=<LoggedModelStatus.READY: 'READY'>, status_message=None),
 LoggedModel(artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/mlruns/1/models/m-f7f945fb7c3b499ab733c9f63026dfab/artifacts', creation_timestamp=1753198953875, experiment_id='1', last_updated_timestamp=1753198958051, model_id='m-f7f945fb7c3b499ab733c9f63026dfab', model_type=None, model_uri='models:/m-f7f945fb7c3b499ab733c9f63026dfab', name='models_mlflow_lr', source_run_id='acb0224c0421483b8a47050d7b77cc23', status=<LoggedModelStatus.READY: 'READY'>, status_message=None),
 LoggedM